In [62]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os

import gmaps
import gmaps.geojson_geometries
import requests

import psycopg2

from geographiclib.geodesic import Geodesic


# Function Code Blocks

### In order to run all of this code, you must have already run project_exercise_3_1.ipynb through project_exercise_3_3.ipynb to have all the necessary data in Postgres/Neo4J

Each of the blocks is a different function used throughout the main block of code

In [63]:
#finds the google api key in order to run the function
file_path = os.path.expanduser('/user/projects/project-3-hahnkenneth/code/gmaps_api_key.txt')
f = open(file_path, 'r')
my_api_key = f.read()
f.close()

In [64]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [65]:
connection = psycopg2.connect(
    user = "",
    password = "",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [66]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [67]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("",""))

In [68]:
session = driver.session(database="neo4j")

In [69]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [70]:
def my_neo4j_shortest_path(from_store, to_customer):
    "given a from store and to customer, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = """
    CALL gds.graph.project(
        'ds_graph', ['Store','Customer','Station'], 
        {
            LINK: {
                type: 'LINK',
                properties: ['weight']
            }
        }
        )
        """
    session.run(query)

    query = """

    MATCH (source:Store {name: $source}), (target:Customer {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_store, target=to_customer)
    paths = {}

    for r in result:
        from_node = r['from']
        to_node = r['to']
        total_cost = r['totalCost']/60
        nodes = r['nodes']
        costs = r['costs']
        paths = {
            'from':from_node,
            'to':to_node,
            'total_cost':total_cost,
            'nodes':nodes,
            'costs':costs
        }
    return paths

In [71]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    MERGE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)

In [72]:
def my_neo4j_create_node_store(store_id,lat_long):
    "create a node with label Store"
    
    query = """
    
    MERGE (c:Store {name: $store_id})
    set c.lat_long = $lat_long
    
    """
    
    session.run(query, store_id=store_id,lat_long = lat_long)

In [73]:
def my_neo4j_create_node_customer(customer_name):
    "create a node with label Customer"
    
    query = """
    
    MERGE (:Customer {name: $customer_name})
    
    """
    
    session.run(query, customer_name=customer_name)

In [74]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    MERGE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [75]:
def my_neo4j_create_relationship_two_way_store(from_store, to_station, weight=0):
    """create relationships two way between a store and a BART station with distance (convert the distance
    into a time it takes to bike that distance)"""
    to_station_adjusted = "depart " + to_station
    query = """
    
    MATCH (from:Store), 
          (to:Station)
    WHERE from.name = $from_store and to.name = $to_station
    MERGE (from)-[:LINK {weight: $weight}]->(to)
    MERGE (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_store=from_store, to_station=to_station_adjusted, weight=weight)

In [76]:
def my_neo4j_create_relationship_two_way_customer(from_customer, to_station, weight):
    """create relationships two way between a customer and a BART station with distance (convert the
    distance into a time it takes to bike that distance)"""
    to_station_adjusted = "arrive " + to_station
    
    query = """
    
    MATCH (from:Customer), 
          (to:Station)
    WHERE from.name = $from_customer and to.name = $to_station
    MERGE (from)-[:LINK {weight: $weight}]->(to)
    MERGE (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_customer=from_customer, to_station=to_station_adjusted, weight=weight)

In [77]:
def my_neo4j_create_relationship_two_way_customer_store(from_store, to_customer, weight):
    "create relationships two way between a store and a customer with distance converted to a travel time (sec)"
    
    query = """
    
    MATCH (from:Store), 
          (to:Customer)
    WHERE from.name = $from_store and to.name = $to_customer
    MERGE (from)-[:LINK {weight: $weight}]->(to)
    MERGE (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_store=from_store, to_customer=to_customer, weight=weight)

In [78]:
def mass_create_relationships_nodes_customers(customers_df):
    """creates all the nodes and relationships for a customers_df dataframe that must include a column called
    nearest_bart (str), full_name (str), and distance_nearest_bart (float)"""
    for index, customer in customers_df.iterrows():
        nearest_bart = customer['nearest_bart']
        customer_id = customer['customer_id']
        distance = customer['nearest_bart_bike_time_seconds']
        my_neo4j_create_node_customer(customer_id)
        my_neo4j_create_relationship_two_way_customer(customer_id,nearest_bart,distance)

In [79]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [80]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [81]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("-------------------------------------------------------------------------------")

In [82]:
def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

In [83]:
def find_nearest_bart(customer_loc,bart):
    """Given a pandas Series of customer longitude-latitude (with each entry in list form [long,lat]),
    and a pandas DataFrame of bart stations with lat_long coordinates (with each entry in list form [long,lat])
    calculate which bart station is the closest from each customer"""
    min_distance = float('inf')
    nearest_station = None
    nearest_station_loc = None
    
    for _, station in bart.iterrows():
        distance = my_calculate_distance(customer_loc,station['lat_long'])
        
        if distance < min_distance:
            min_distance = distance
            nearest_station = station['station']
            nearest_station_loc = station['lat_long']
    return nearest_station,nearest_station_loc

In [84]:
def find_nearest_store(customer_loc,store):
    """Given a pandas Series of customer longitude-latitude (with each entry in list form [long,lat]),
    and a pandas DataFrame of stores with lat_long coordinates (with each entry in list form [long,lat])
    calculate which store is the closest from each customer"""
    min_distance = float('inf')
    nearest_station = None
    nearest_station_loc = None
    
    for _, store in store.iterrows():
        distance = my_calculate_distance(customer_loc,store['lat_long'])
        
        if distance < min_distance:
            min_distance = distance
            nearest_store = store['store_id']
            nearest_store_loc = store['lat_long']
    return nearest_store,nearest_store_loc

In [85]:
def my_neo4j_delete_node(station_name):
    "delete a node with label Station (in case you accidentally made a new node)"
    
    query = """
    
    MERGE (n:Station {name: $station_name})
    DETACH DELETE n
    
    """
    
    session.run(query, station_name=station_name)

In [86]:
def my_neo4j_delete_relationship_two_way_store(from_store, to_station):
    "delete relationships two way between a store and a BART station with distance"
    
    query1 = """
    
    MATCH (from:Store)-[r1:LINK]->(to:Station)
    WHERE from.name = $from_store and to.name = $to_station
    delete r1
    
    """
    
    query2 = """
    
    MATCH (to:Station)-[r2:LINK]->(from:Store)
    WHERE from.name = $from_store AND to.name = $to_station
    DELETE r2
    
    """
    
    session.run(query1, from_store=from_store, to_station=to_station)
    session.run(query2, from_store=from_store, to_station=to_station)

In [87]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [88]:
def google_api_bike_travel_time(api_key, origin, destination):
    """Use Google Maps API for a given key to get the biking travel time between the origin [lat,long]
    and the destination [lat,long]. Divides  that travel time by the e_bike_factor, correcting for the speed
    of an e-bike over normal biking."""
    base_url = "https://maps.googleapis.com/maps/api/directions/json?"
    params = {
        'origin': f"{origin[0]},{origin[1]}",  # Latitude, Longitude
        'destination': f"{destination[0]},{destination[1]}",  # Latitude, Longitude
        'mode': 'bicycling',
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    directions = response.json()
    
    e_bike_factor = 1.5
    
    if directions['status'] == 'OK':
        route = directions['routes'][0]
        leg = route['legs'][0]
        duration = leg['duration']['value'] 
        return duration / e_bike_factor
    else:
        return "Invalid Location"

In [89]:
def find_biking_time(thing,store_or_bart="bart",manual=0):
    """Used to find conduct the API call to find the biking times between a thing and either to the bart or to
    the store. Only used for the .apply() function on a pandas dataframe."""
    if store_or_bart == "bart":
        origin = thing['lat_long']
        destination = thing['nearest_bart_lat_long']
    elif store_or_bart == "store":
        origin = thing['lat_long']
        destination = thing['nearest_store_lat_long']
    return google_api_bike_travel_time(my_api_key,origin,destination)

In [90]:
def my_neo4j_update_thing_lat_long(row,thing):
    """use the apply function on a pandas dataframe along with the thing which is a string that can either be
    station, store, or customer"""
    if thing == 'station':
        query = """

        MATCH (c:Station {name: $station_name})
        SET c.lat_long = $lat_long

        """
        session.run(query, station_name = 'depart '+row['station'],lat_long=row['lat_long'])
        session.run(query, station_name = 'arrive '+row['station'],lat_long=row['lat_long'])
    elif thing == 'store':
        query = """

        MATCH (c:Store {name: $store_name})
        SET c.lat_long = $lat_long

        """
        session.run(query, store_name = row['store_id'],lat_long=row['lat_long'])
    elif thing == 'customer':
        query = """

        MATCH (c:Customer {name: $customer_name})
        SET c.lat_long = $lat_long

        """
        session.run(query, customer_name = row['customer_id'],lat_long=row['lat_long'])

In [91]:
def create_new_store_with_relationships(store_name):
    """This function will create a new store node if input a string that is the same as any of the bart stations.
    It will connect the new store node to that bart station with weight 0 and it will also create relationships
    between the new store and all the customers based on the biking time"""
    store = {'store_id': f'Store {store_name}',
                   'lat_long': stations_df.loc[stations_df['station']==f'{store_name}','lat_long'].values[0]}
    store_customers_df = customers_small_df[['customer_id','lat_long']]
    store_customers_df['nearest_store'] = f'Store {store_name}'
    store_customers_df['nearest_store_lat_long'] = [store['lat_long']] * len(store_customers_df)
    my_neo4j_create_node_store(store['store_id'],store['lat_long'])
    my_neo4j_create_relationship_two_way_store(store['store_id'],store_name)
    store_customers_df['nearest_store_bike_time_seconds'] = store_customers_df.apply(lambda row: find_biking_time(row,"store"),axis=1)
    
    for index, customer in store_customers_df.iterrows():
        nearest_store = customer['nearest_store']
        customer_name = customer['customer_id']
        weight = customer['nearest_store_bike_time_seconds']
        my_neo4j_create_relationship_two_way_customer_store(nearest_store,customer_name,weight)

In [92]:
def get_nodes(session, label):
    """Get all the nodes of a certain label for a neo4j session, return as a list"""
    query = f"MATCH (n:{label}) RETURN n.name AS name,n.lat_long AS lat_long"
    result = session.run(query)
    node_dict = {}
    return [row['name'] for row in result]

# Main Code Blocks

## Querying blocks

Query all customers, stores, and stations in California Bay Area

In [93]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select
    c.customer_id,
    c.first_name,
    c.last_name,
    c.city,
    c.state,
    c.zip,
    z.latitude,
    z.longitude,
    z.area
from customers c
join zip_codes z on
    c.zip = z.zip
where c.state = 'CA'

"""

# query for all customers in Califronia
customers_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# get the area of the customer's location and sqrt to get the side lengths (assuming it's a square)
customers_df['zip_box_miles'] = np.sqrt(customers_df['area'])

# combine latitude and longitude into one list to apply into my_calculate_box
customers_df['zip_center_point'] = customers_df.apply(lambda row: [row['latitude'],row['longitude']],axis=1)


# create a geographical box for each customer with the centerpoint being the [lat,long] of the customer
# and the size of the box as half the diagonal of the side length sqrt(2)/2 is half the diagonal of a square.
customers_df['customer_box'] = customers_df.apply(lambda row: my_calculate_box(row['zip_center_point']
                                                                               ,row['zip_box_miles']*np.sqrt(2)*0.5),axis=1)
customers_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box
0,1,Robb,Weaving,Oakland,CA,94609,37.8343,-122.2643,1.8920,1.375500,"[37.8343, -122.2643]","((37.83429865773755, -122.2820813217889), (37...."
1,2,Robby,Belliard,Oakland,CA,94609,37.8343,-122.2643,1.8920,1.375500,"[37.8343, -122.2643]","((37.83429865773755, -122.2820813217889), (37...."
2,3,Sadella,Caudrelier,Oakland,CA,94609,37.8343,-122.2643,1.8920,1.375500,"[37.8343, -122.2643]","((37.83429865773755, -122.2820813217889), (37...."
3,4,Holmes,Shimmings,Oakland,CA,94609,37.8343,-122.2643,1.8920,1.375500,"[37.8343, -122.2643]","((37.83429865773755, -122.2820813217889), (37...."
4,5,Beverley,Gubbin,Oakland,CA,94609,37.8343,-122.2643,1.8920,1.375500,"[37.8343, -122.2643]","((37.83429865773755, -122.2820813217889), (37...."
...,...,...,...,...,...,...,...,...,...,...,...,...
8133,8134,Wilmette,Raittie,San Geronimo,CA,94963,38.0138,-122.6703,2.3307,1.526663,"[38.0138, -122.6703]","((38.01379833585439, -122.69008346421008), (38..."
8134,8135,Ammamaria,Newens,San Geronimo,CA,94963,38.0138,-122.6703,2.3307,1.526663,"[38.0138, -122.6703]","((38.01379833585439, -122.69008346421008), (38..."
8135,8136,Blakeley,Verry,San Geronimo,CA,94963,38.0138,-122.6703,2.3307,1.526663,"[38.0138, -122.6703]","((38.01379833585439, -122.69008346421008), (38..."
8136,8137,Kikelia,Mendus,San Geronimo,CA,94963,38.0138,-122.6703,2.3307,1.526663,"[38.0138, -122.6703]","((38.01379833585439, -122.69008346421008), (38..."


In [94]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select
    station,
    latitude,
    longitude
from stations
order by latitude,longitude

"""

# query all the bart stations and combine into a lat_long [lat,long] column
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations_df['lat_long'] = stations_df.apply(lambda row: [row['latitude'],row['longitude']],axis=1)
stations_df

,station,latitude,longitude,lat_long
0,Berryessa,37.368361,-121.874655,"[37.368361, -121.874655]"
1,Milpitas,37.410278,-121.891111,"[37.410278, -121.891111]"
2,Warm Springs,37.503000,-121.940000,"[37.503, -121.94]"
3,Fremont,37.557489,-121.976620,"[37.557489, -121.97662]"
4,Union City,37.590746,-122.017282,"[37.590746, -122.017282]"
5,Millbrae,37.600300,-122.386700,"[37.6003, -122.3867]"
6,SFO,37.616400,-122.391000,"[37.6164, -122.391]"
7,South Hayward,37.634362,-122.057172,"[37.634362, -122.057172]"
8,San Bruno,37.638300,-122.416500,"[37.6383, -122.4165]"
9,South San Francisco,37.664264,-122.444043,"[37.664264, -122.444043]"


In [95]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select
    *
from stores
where state = 'CA'

"""

# query for all stores in California
stores_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

# combine the latitude and longitude into one column
stores_df['lat_long'] = stores_df.apply(lambda row: [row['latitude'],row['longitude']],axis=1)
stores_df['store_id'] = 'Store' + ' ' + stores_df['store_id'].astype(str)
stores_df

,store_id,street,city,state,zip,latitude,longitude,lat_long
0,Store 1,3000 Telegraph Ave,Berkeley,CA,94705,37.8555,-122.2604,"[37.8555, -122.2604]"


## Customer DataFrame Manipulation

Determine latitude and longitude of customers using a "box" and finding random lat_long coordinates within the box.

Also find the nearest bart station to each of the customers.

Finally, use Google Maps Direction API to find travel time between BART and Customer.

In [96]:
# create random seed and for each customer randomly determine coordinates within the box made in the prior code
random.seed(205)
customers_df['lat_long'] = customers_df.apply(lambda row: (random.uniform(row['customer_box'][3][0]
                                                                                ,row['customer_box'][2][0])
                                                                 ,random.uniform(row['customer_box'][0][1]
                                                                               ,row['customer_box'][1][1]))
                                                     ,axis = 1) 

# get 50 random customers so the code doesn't take that long to run
customers_small_df = customers_df.sample(n=50,random_state=205)
customers_small_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box,lat_long
7868,7869,Phedra,Spracklin,Castro Valley,CA,94552,37.7131,-122.0180,47.9115,6.921813,"[37.7131, -122.018]","((37.713066156874426, -122.10733348806473), (3...","(37.669152243729144, -121.98737165028699)"
7004,7005,Clayborn,Weiss,Alamo,CA,94507,37.8504,-122.0211,13.3413,3.652574,"[37.8504, -122.0211]","((37.850390529678364, -122.06832771428171), (3...","(37.835352838864466, -122.06589003386509)"
7177,7178,Camel,Mertgen,Brisbane,CA,94005,37.6887,-122.4080,4.8168,2.194721,"[37.6887, -122.408]","((37.688696600542514, -122.4363159726673), (37...","(37.700974583746536, -122.38461871603742)"
736,737,Lindsy,Bonass,Berkeley,CA,94705,37.8652,-122.2382,3.4614,1.860484,"[37.8652, -122.2382]","((37.86519754162312, -122.26226083019446), (37...","(37.85323476735731, -122.24807899866538)"
1563,1564,Aluin,Armsby,Oakland,CA,94612,37.8088,-122.2691,0.9543,0.976883,"[37.8088, -122.2691]","((37.808799323598265, -122.28172398655485), (3...","(37.8016718414982, -122.26567866295639)"
1404,1405,Che,Sellens,Oakland,CA,94610,37.8114,-122.2423,2.3466,1.531862,"[37.8114, -122.2423]","((37.811398336590074, -122.26209651845227), (3...","(37.82699612577277, -122.23158868586403)"
1947,1948,Trixi,Laying,Berkeley,CA,94707,37.8983,-122.2792,1.7251,1.313431,"[37.8983, -122.2792]","((37.89829877333669, -122.29619363268857), (37...","(37.89505795681284, -122.28409059113466)"
7069,7070,Becca,Kesten,San Leandro,CA,94578,37.7064,-122.1252,4.7800,2.186321,"[37.7064, -122.1252]","((37.70639662437308, -122.15341430614241), (37...","(37.726533436025, -122.12133024564147)"
7638,7639,Wilbert,Hew,San Francisco,CA,94132,37.7222,-122.4849,3.4813,1.865824,"[37.7222, -122.4849]","((37.72219754011997, -122.50898343200411), (37...","(37.716480882546826, -122.50255973605196)"
3993,3994,Ambur,Bridgestock,San Francisco,CA,94130,37.8205,-122.3695,0.9371,0.968039,"[37.8205, -122.3695]","((37.820499335511194, -122.38201167829628), (3...","(37.82437228445577, -122.35949345559403)"


In [97]:
# for each customer, find the nearest bart station along with the distance to the bart (SLOW)
customers_small_df[['nearest_bart','nearest_bart_lat_long']] = customers_small_df['lat_long'].apply(
    lambda loc: pd.Series(find_nearest_bart(loc,stations_df)))

# modify customer_id column to make customer_id easily identifiable
# makes it easier to see on the neo4j graph
customers_small_df['customer_id'] = 'Customer' + ' ' + customers_small_df['customer_id'].astype(str)
customers_small_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box,lat_long,nearest_bart,nearest_bart_lat_long
7868,Customer 7869,Phedra,Spracklin,Castro Valley,CA,94552,37.7131,-122.0180,47.9115,6.921813,"[37.7131, -122.018]","((37.713066156874426, -122.10733348806473), (3...","(37.669152243729144, -121.98737165028699)",West Dublin,"[37.699726, -121.928273]"
7004,Customer 7005,Clayborn,Weiss,Alamo,CA,94507,37.8504,-122.0211,13.3413,3.652574,"[37.8504, -122.0211]","((37.850390529678364, -122.06832771428171), (3...","(37.835352838864466, -122.06589003386509)",Walnut Creek,"[37.905724, -122.067332]"
7177,Customer 7178,Camel,Mertgen,Brisbane,CA,94005,37.6887,-122.4080,4.8168,2.194721,"[37.6887, -122.408]","((37.688696600542514, -122.4363159726673), (37...","(37.700974583746536, -122.38461871603742)",Glen Park,"[37.733118, -122.433808]"
736,Customer 737,Lindsy,Bonass,Berkeley,CA,94705,37.8652,-122.2382,3.4614,1.860484,"[37.8652, -122.2382]","((37.86519754162312, -122.26226083019446), (37...","(37.85323476735731, -122.24807899866538)",Rockridge,"[37.844452, -122.252083]"
1563,Customer 1564,Aluin,Armsby,Oakland,CA,94612,37.8088,-122.2691,0.9543,0.976883,"[37.8088, -122.2691]","((37.808799323598265, -122.28172398655485), (3...","(37.8016718414982, -122.26567866295639)",Lake Merritt,"[37.797773, -122.266588]"
1404,Customer 1405,Che,Sellens,Oakland,CA,94610,37.8114,-122.2423,2.3466,1.531862,"[37.8114, -122.2423]","((37.811398336590074, -122.26209651845227), (3...","(37.82699612577277, -122.23158868586403)",Rockridge,"[37.844452, -122.252083]"
1947,Customer 1948,Trixi,Laying,Berkeley,CA,94707,37.8983,-122.2792,1.7251,1.313431,"[37.8983, -122.2792]","((37.89829877333669, -122.29619363268857), (37...","(37.89505795681284, -122.28409059113466)",El Cerrito Plaza,"[37.902694, -122.298968]"
7069,Customer 7070,Becca,Kesten,San Leandro,CA,94578,37.7064,-122.1252,4.7800,2.186321,"[37.7064, -122.1252]","((37.70639662437308, -122.15341430614241), (37...","(37.726533436025, -122.12133024564147)",Bay Fair,"[37.697, -122.1265]"
7638,Customer 7639,Wilbert,Hew,San Francisco,CA,94132,37.7222,-122.4849,3.4813,1.865824,"[37.7222, -122.4849]","((37.72219754011997, -122.50898343200411), (37...","(37.716480882546826, -122.50255973605196)",Daly City,"[37.706224, -122.468934]"
3993,Customer 3994,Ambur,Bridgestock,San Francisco,CA,94130,37.8205,-122.3695,0.9371,0.968039,"[37.8205, -122.3695]","((37.820499335511194, -122.38201167829628), (3...","(37.82437228445577, -122.35949345559403)",Embarcadero,"[37.793056, -122.397222]"


In [98]:
customers_small_df['nearest_bart_bike_time_seconds'] = customers_small_df.apply(lambda row: find_biking_time(row,"bart"),axis=1)

customers_small_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box,lat_long,nearest_bart,nearest_bart_lat_long,nearest_bart_bike_time_seconds
7868,Customer 7869,Phedra,Spracklin,Castro Valley,CA,94552,37.7131,-122.0180,47.9115,6.921813,"[37.7131, -122.018]","((37.713066156874426, -122.10733348806473), (3...","(37.669152243729144, -121.98737165028699)",West Dublin,"[37.699726, -121.928273]",1951.333333
7004,Customer 7005,Clayborn,Weiss,Alamo,CA,94507,37.8504,-122.0211,13.3413,3.652574,"[37.8504, -122.0211]","((37.850390529678364, -122.06832771428171), (3...","(37.835352838864466, -122.06589003386509)",Walnut Creek,"[37.905724, -122.067332]",2876.666667
7177,Customer 7178,Camel,Mertgen,Brisbane,CA,94005,37.6887,-122.4080,4.8168,2.194721,"[37.6887, -122.408]","((37.688696600542514, -122.4363159726673), (37...","(37.700974583746536, -122.38461871603742)",Glen Park,"[37.733118, -122.433808]",1390.666667
736,Customer 737,Lindsy,Bonass,Berkeley,CA,94705,37.8652,-122.2382,3.4614,1.860484,"[37.8652, -122.2382]","((37.86519754162312, -122.26226083019446), (37...","(37.85323476735731, -122.24807899866538)",Rockridge,"[37.844452, -122.252083]",235.333333
1563,Customer 1564,Aluin,Armsby,Oakland,CA,94612,37.8088,-122.2691,0.9543,0.976883,"[37.8088, -122.2691]","((37.808799323598265, -122.28172398655485), (3...","(37.8016718414982, -122.26567866295639)",Lake Merritt,"[37.797773, -122.266588]",107.333333
1404,Customer 1405,Che,Sellens,Oakland,CA,94610,37.8114,-122.2423,2.3466,1.531862,"[37.8114, -122.2423]","((37.811398336590074, -122.26209651845227), (3...","(37.82699612577277, -122.23158868586403)",Rockridge,"[37.844452, -122.252083]",648.666667
1947,Customer 1948,Trixi,Laying,Berkeley,CA,94707,37.8983,-122.2792,1.7251,1.313431,"[37.8983, -122.2792]","((37.89829877333669, -122.29619363268857), (37...","(37.89505795681284, -122.28409059113466)",El Cerrito Plaza,"[37.902694, -122.298968]",262.000000
7069,Customer 7070,Becca,Kesten,San Leandro,CA,94578,37.7064,-122.1252,4.7800,2.186321,"[37.7064, -122.1252]","((37.70639662437308, -122.15341430614241), (37...","(37.726533436025, -122.12133024564147)",Bay Fair,"[37.697, -122.1265]",844.000000
7638,Customer 7639,Wilbert,Hew,San Francisco,CA,94132,37.7222,-122.4849,3.4813,1.865824,"[37.7222, -122.4849]","((37.72219754011997, -122.50898343200411), (37...","(37.716480882546826, -122.50255973605196)",Daly City,"[37.706224, -122.468934]",884.666667
3993,Customer 3994,Ambur,Bridgestock,San Francisco,CA,94130,37.8205,-122.3695,0.9371,0.968039,"[37.8205, -122.3695]","((37.820499335511194, -122.38201167829628), (3...","(37.82437228445577, -122.35949345559403)",Embarcadero,"[37.793056, -122.397222]",4958.666667


## Stores DataFrame Manipulation

Find the nearest bart station to each of the stores (only one store at the moment).

Finally, use Google Maps Direction API to find travel time between BART and Store.

In [99]:
# find the nearest bart station to the stores in California
stores_df[['nearest_bart','nearest_bart_lat_long']] = stores_df['lat_long'].apply(
    lambda loc: pd.Series(find_nearest_bart(loc,stations_df)))

stores_df

,store_id,street,city,state,zip,latitude,longitude,lat_long,nearest_bart,nearest_bart_lat_long
0,Store 1,3000 Telegraph Ave,Berkeley,CA,94705,37.8555,-122.2604,"[37.8555, -122.2604]",Ashby,"[37.853068, -122.269957]"


In [100]:
stores_df['nearest_bart_bike_time_seconds'] = stores_df.apply(lambda row: find_biking_time(row,"bart"),axis=1)

stores_df

,store_id,street,city,state,zip,latitude,longitude,lat_long,nearest_bart,nearest_bart_lat_long,nearest_bart_bike_time_seconds
0,Store 1,3000 Telegraph Ave,Berkeley,CA,94705,37.8555,-122.2604,"[37.8555, -122.2604]",Ashby,"[37.853068, -122.269957]",236.666667


## Neo4j Add Nodes and Relationships

In [101]:
# see the number of nodes and relationships before adding new ones (should be 214 nodes and 652 relationships)
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


Add nodes for each store and the travel time from the store to the nearest bart station

In [102]:
# add the stores as nodes and the relationship as the time it takes to bike to the nearest bart station
for index, store in stores_df.iterrows():
    my_neo4j_create_node_store(store['store_id'],store['lat_long'])
    
    nearest_bart = store['nearest_bart']
    
    my_neo4j_create_relationship_two_way_store(store['store_id'],nearest_bart,store['nearest_bart_bike_time_seconds'])

In [103]:
# in this case, there is only one store and two relationships added
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 215
  Relationships: 654
-------------------------


add relationship to the customer and each bart station that they're nearest to

In [104]:
# for every customer, create nodes for each customer_id and create relationships between each customer and 
# their respective closest bart station, with the weight being the time it takes to bike to the nearest bart
mass_create_relationships_nodes_customers(customers_small_df)

In [105]:
# Nodes: 265
# Relationships: 754
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 265
  Relationships: 754
-------------------------


## Customer and Store Relationship

For each customer, find the travel distance between a customer and a store using API call to Google Directions

In [106]:
# Find the nearest store to each customer (because there is only one store, it will be the same for each customer
# but the distance will be different for each
# I just did it this way so if we add more stores, we can still run this code
customers_small_df[['nearest_store','nearest_store_lat_long']] = customers_small_df['lat_long'].apply(
    lambda loc: pd.Series(find_nearest_store(loc,stores_df)))
    
customers_small_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box,lat_long,nearest_bart,nearest_bart_lat_long,nearest_bart_bike_time_seconds,nearest_store,nearest_store_lat_long
7868,Customer 7869,Phedra,Spracklin,Castro Valley,CA,94552,37.7131,-122.0180,47.9115,6.921813,"[37.7131, -122.018]","((37.713066156874426, -122.10733348806473), (3...","(37.669152243729144, -121.98737165028699)",West Dublin,"[37.699726, -121.928273]",1951.333333,Store 1,"[37.8555, -122.2604]"
7004,Customer 7005,Clayborn,Weiss,Alamo,CA,94507,37.8504,-122.0211,13.3413,3.652574,"[37.8504, -122.0211]","((37.850390529678364, -122.06832771428171), (3...","(37.835352838864466, -122.06589003386509)",Walnut Creek,"[37.905724, -122.067332]",2876.666667,Store 1,"[37.8555, -122.2604]"
7177,Customer 7178,Camel,Mertgen,Brisbane,CA,94005,37.6887,-122.4080,4.8168,2.194721,"[37.6887, -122.408]","((37.688696600542514, -122.4363159726673), (37...","(37.700974583746536, -122.38461871603742)",Glen Park,"[37.733118, -122.433808]",1390.666667,Store 1,"[37.8555, -122.2604]"
736,Customer 737,Lindsy,Bonass,Berkeley,CA,94705,37.8652,-122.2382,3.4614,1.860484,"[37.8652, -122.2382]","((37.86519754162312, -122.26226083019446), (37...","(37.85323476735731, -122.24807899866538)",Rockridge,"[37.844452, -122.252083]",235.333333,Store 1,"[37.8555, -122.2604]"
1563,Customer 1564,Aluin,Armsby,Oakland,CA,94612,37.8088,-122.2691,0.9543,0.976883,"[37.8088, -122.2691]","((37.808799323598265, -122.28172398655485), (3...","(37.8016718414982, -122.26567866295639)",Lake Merritt,"[37.797773, -122.266588]",107.333333,Store 1,"[37.8555, -122.2604]"
1404,Customer 1405,Che,Sellens,Oakland,CA,94610,37.8114,-122.2423,2.3466,1.531862,"[37.8114, -122.2423]","((37.811398336590074, -122.26209651845227), (3...","(37.82699612577277, -122.23158868586403)",Rockridge,"[37.844452, -122.252083]",648.666667,Store 1,"[37.8555, -122.2604]"
1947,Customer 1948,Trixi,Laying,Berkeley,CA,94707,37.8983,-122.2792,1.7251,1.313431,"[37.8983, -122.2792]","((37.89829877333669, -122.29619363268857), (37...","(37.89505795681284, -122.28409059113466)",El Cerrito Plaza,"[37.902694, -122.298968]",262.000000,Store 1,"[37.8555, -122.2604]"
7069,Customer 7070,Becca,Kesten,San Leandro,CA,94578,37.7064,-122.1252,4.7800,2.186321,"[37.7064, -122.1252]","((37.70639662437308, -122.15341430614241), (37...","(37.726533436025, -122.12133024564147)",Bay Fair,"[37.697, -122.1265]",844.000000,Store 1,"[37.8555, -122.2604]"
7638,Customer 7639,Wilbert,Hew,San Francisco,CA,94132,37.7222,-122.4849,3.4813,1.865824,"[37.7222, -122.4849]","((37.72219754011997, -122.50898343200411), (37...","(37.716480882546826, -122.50255973605196)",Daly City,"[37.706224, -122.468934]",884.666667,Store 1,"[37.8555, -122.2604]"
3993,Customer 3994,Ambur,Bridgestock,San Francisco,CA,94130,37.8205,-122.3695,0.9371,0.968039,"[37.8205, -122.3695]","((37.820499335511194, -122.38201167829628), (3...","(37.82437228445577, -122.35949345559403)",Embarcadero,"[37.793056, -122.397222]",4958.666667,Store 1,"[37.8555, -122.2604]"


In [107]:
customers_small_df['nearest_store_bike_time_seconds'] = customers_small_df.apply(lambda row: find_biking_time(row,"store"),axis=1)

customers_small_df

,customer_id,first_name,last_name,city,state,zip,latitude,longitude,area,zip_box_miles,zip_center_point,customer_box,lat_long,nearest_bart,nearest_bart_lat_long,nearest_bart_bike_time_seconds,nearest_store,nearest_store_lat_long,nearest_store_bike_time_seconds
7868,Customer 7869,Phedra,Spracklin,Castro Valley,CA,94552,37.7131,-122.0180,47.9115,6.921813,"[37.7131, -122.018]","((37.713066156874426, -122.10733348806473), (3...","(37.669152243729144, -121.98737165028699)",West Dublin,"[37.699726, -121.928273]",1951.333333,Store 1,"[37.8555, -122.2604]",5482.000000
7004,Customer 7005,Clayborn,Weiss,Alamo,CA,94507,37.8504,-122.0211,13.3413,3.652574,"[37.8504, -122.0211]","((37.850390529678364, -122.06832771428171), (3...","(37.835352838864466, -122.06589003386509)",Walnut Creek,"[37.905724, -122.067332]",2876.666667,Store 1,"[37.8555, -122.2604]",5058.666667
7177,Customer 7178,Camel,Mertgen,Brisbane,CA,94005,37.6887,-122.4080,4.8168,2.194721,"[37.6887, -122.408]","((37.688696600542514, -122.4363159726673), (37...","(37.700974583746536, -122.38461871603742)",Glen Park,"[37.733118, -122.433808]",1390.666667,Store 1,"[37.8555, -122.2604]",5057.333333
736,Customer 737,Lindsy,Bonass,Berkeley,CA,94705,37.8652,-122.2382,3.4614,1.860484,"[37.8652, -122.2382]","((37.86519754162312, -122.26226083019446), (37...","(37.85323476735731, -122.24807899866538)",Rockridge,"[37.844452, -122.252083]",235.333333,Store 1,"[37.8555, -122.2604]",240.000000
1563,Customer 1564,Aluin,Armsby,Oakland,CA,94612,37.8088,-122.2691,0.9543,0.976883,"[37.8088, -122.2691]","((37.808799323598265, -122.28172398655485), (3...","(37.8016718414982, -122.26567866295639)",Lake Merritt,"[37.797773, -122.266588]",107.333333,Store 1,"[37.8555, -122.2604]",1079.333333
1404,Customer 1405,Che,Sellens,Oakland,CA,94610,37.8114,-122.2423,2.3466,1.531862,"[37.8114, -122.2423]","((37.811398336590074, -122.26209651845227), (3...","(37.82699612577277, -122.23158868586403)",Rockridge,"[37.844452, -122.252083]",648.666667,Store 1,"[37.8555, -122.2604]",894.000000
1947,Customer 1948,Trixi,Laying,Berkeley,CA,94707,37.8983,-122.2792,1.7251,1.313431,"[37.8983, -122.2792]","((37.89829877333669, -122.29619363268857), (37...","(37.89505795681284, -122.28409059113466)",El Cerrito Plaza,"[37.902694, -122.298968]",262.000000,Store 1,"[37.8555, -122.2604]",1122.666667
7069,Customer 7070,Becca,Kesten,San Leandro,CA,94578,37.7064,-122.1252,4.7800,2.186321,"[37.7064, -122.1252]","((37.70639662437308, -122.15341430614241), (37...","(37.726533436025, -122.12133024564147)",Bay Fair,"[37.697, -122.1265]",844.000000,Store 1,"[37.8555, -122.2604]",3293.333333
7638,Customer 7639,Wilbert,Hew,San Francisco,CA,94132,37.7222,-122.4849,3.4813,1.865824,"[37.7222, -122.4849]","((37.72219754011997, -122.50898343200411), (37...","(37.716480882546826, -122.50255973605196)",Daly City,"[37.706224, -122.468934]",884.666667,Store 1,"[37.8555, -122.2604]",6236.000000
3993,Customer 3994,Ambur,Bridgestock,San Francisco,CA,94130,37.8205,-122.3695,0.9371,0.968039,"[37.8205, -122.3695]","((37.820499335511194, -122.38201167829628), (3...","(37.82437228445577, -122.35949345559403)",Embarcadero,"[37.793056, -122.397222]",4958.666667,Store 1,"[37.8555, -122.2604]",2500.000000


The below code creates a relationship between all the customers and the nearest store. This way, if a person is already near a store, then the shortest path is going to be to bike to the store instead of biking to a bart station and biking back to a customer.

In [108]:
for index, customer in customers_small_df.iterrows():
    nearest_store = customer['nearest_store']
    customer_name = customer['customer_id']
    weight = customer['nearest_store_bike_time_seconds']
    my_neo4j_create_relationship_two_way_customer_store(nearest_store,customer_name,weight)

In [109]:
# Nodes: 265
# Relationships: 854
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 265
  Relationships: 854
-------------------------


## Find Shortest Path between all Stores and all Customers

Use Neo4j to find shortest path

In [110]:
stores = get_nodes(session, "Store")
customers = get_nodes(session, "Customer")
stores

['Store 1']

In [111]:
# Create a DataFrame to store results
df = pd.DataFrame(columns=['store', 'customer', 'totalCost', 'nodes', 'costs'])
len_customers = len(customers)
# Loop through all stores and customers and get the shortest path from all stores to all customers
for store in stores:
    for customer in customers:
        result = my_neo4j_shortest_path(store, customer)
        if result:
            df = df.append({
                'store': result['from'],
                'customer': result['to'],
                'totalCost': result['total_cost'],
                'nodes': result['nodes'],
                'costs': result['costs']
            }, ignore_index=True)

In [112]:
df.sort_values('totalCost',ascending=False)

,store,customer,totalCost,nodes,costs
0,Store 1,Customer 7869,80.366667,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ..."
17,Store 1,Customer 6697,80.261111,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ..."
1,Store 1,Customer 7005,75.872222,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ..."
40,Store 1,Customer 5239,62.733333,"[Store 1, Customer 5239]","[0.0, 3764.0]"
49,Store 1,Customer 6752,61.644444,"[Store 1, depart Ashby, red Ashby, red Downtow...","[0.0, 236.66666666666666, 236.66666666666666, ..."
25,Store 1,Customer 7246,61.200000,"[Store 1, depart Ashby, red Ashby, red MacArth...","[0.0, 236.66666666666666, 236.66666666666666, ..."
2,Store 1,Customer 7178,60.122222,"[Store 1, depart Ashby, red Ashby, red MacArth...","[0.0, 236.66666666666666, 236.66666666666666, ..."
22,Store 1,Customer 7913,59.855556,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ..."
8,Store 1,Customer 7639,57.688889,"[Store 1, depart Ashby, red Ashby, red MacArth...","[0.0, 236.66666666666666, 236.66666666666666, ..."
24,Store 1,Customer 7771,57.361111,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ..."


The block below finds the last node before reaching the customer. This allows us to see which BART station is the furthest away from the Ashby Bart Station and how long the total cost was. The below code also finds what the fastest route was from each store to each customer.

In [113]:
df['finalnode'] = df['nodes'].apply(lambda row: row[-2])
df = df.sort_values('totalCost',ascending=False)

min_cost_indices = df.groupby('customer')['totalCost'].idxmin()
min_cost_df = df.loc[min_cost_indices]
min_cost_df

,store,customer,totalCost,nodes,costs,finalnode
43,Store 1,Customer 1224,9.833333,"[Store 1, Customer 1224]","[0.0, 590.0]",Store 1
48,Store 1,Customer 1374,14.055556,"[Store 1, depart Ashby, orange Ashby, orange D...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive North Berkeley
46,Store 1,Customer 1380,12.933333,"[Store 1, depart Ashby, orange Ashby, orange D...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive North Berkeley
44,Store 1,Customer 139,8.577778,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive MacArthur
5,Store 1,Customer 1405,14.900000,"[Store 1, Customer 1405]","[0.0, 894.0]",Store 1
4,Store 1,Customer 1564,17.733333,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive Lake Merritt
6,Store 1,Customer 1948,16.311111,"[Store 1, depart Ashby, red Ashby, red Downtow...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive El Cerrito Plaza
20,Store 1,Customer 2429,18.744444,"[Store 1, depart Ashby, orange Ashby, orange M...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive Lake Merritt
45,Store 1,Customer 253,2.711111,"[Store 1, Customer 253]","[0.0, 162.66666666666666]",Store 1
15,Store 1,Customer 2943,38.422222,"[Store 1, depart Ashby, red Ashby, red Downtow...","[0.0, 236.66666666666666, 236.66666666666666, ...",arrive El Cerrito del Norte


The below code groups the stations together to find the furthest BART stations in terms of mean travel time. We are interested in mainly travel times greater than 40 minutes

In [114]:
result_df = min_cost_df.groupby('finalnode')['totalCost'].agg(['mean','count']).reset_index()
result_df.columns = ['finalnode','mean_travel_time_minutes','count_customers']
result_df['percent_customers'] = result_df['count_customers']/len(min_cost_df)*100
result_df = result_df.sort_values('mean_travel_time_minutes',ascending=False)
impacted_customers_df = result_df[result_df['mean_travel_time_minutes'] > 40]
impacted_customers_df

,finalnode,mean_travel_time_minutes,count_customers,percent_customers
22,arrive West Dublin,80.366667,1,2.0
17,arrive North Concord,80.261111,1,2.0
20,arrive Richmond,61.644444,1,2.0
13,arrive Hayward,59.855556,1,2.0
7,arrive Daly City,57.688889,1,2.0
12,arrive Glen Park,57.266667,2,4.0
21,arrive Walnut Creek,54.536111,4,8.0
3,arrive Balboa Park,52.838889,2,4.0
6,arrive Concord,50.527778,1,2.0
4,arrive Bay Fair,47.011111,1,2.0


## Louvain Modularity

Use Louvain Modularity to find clusters for each station to make it easier to find communities.

In [115]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})
"""

session.run(query)

In [116]:
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

station_modularity = my_neo4j_run_query_pandas(query)

In [117]:
filtered_station_modularity = station_modularity[~station_modularity['name'].str.contains('arrive|depart',regex=True)]

In [118]:
filtered_station_modularity['community'].value_counts()

47    13
29    12
31    12
59    12
51    11
63    10
91    10
67     9
85     9
7      8
21     8
Name: community, dtype: int64

In [119]:
# because we have multiple nodes for each station, create a new column that is just the name of the station
filtered_station_modularity['station'] = filtered_station_modularity['name'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# group by the community in order to get one community number for each station name
grouped_station_modularity = filtered_station_modularity.groupby('station')['community'].max().reset_index()

/tmp/ipykernel_42/1524616281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_station_modularity['station'] = filtered_station_modularity['name'].apply(lambda x: ' '.join(x.split(' ')[1:]))


In [120]:
grouped_station_modularity.sort_values('community')

,station,community
1,16th Street Mission,7
3,24th Street Mission,7
10,Civic Center,21
37,Powell Street,21
6,Balboa Park,29
22,Glen Park,29
14,Daly City,29
5,Ashby,31
18,El Cerrito del Norte,31
17,El Cerrito Plaza,31


## Betweenness Centrality

Use betweenness centrality to find stations with the lowest betweenness

In [121]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', ['Station','Customer','Store'], 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

In [122]:
query = """

CALL gds.betweenness.stream('ds_graph', {relationshipWeightProperty: 'weight'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

betweenness_df = my_neo4j_run_query_pandas(query)
betweenness_df

,name,betweenness
0,Store 1,16553.000000
1,depart Ashby,14787.000000
2,orange MacArthur,13726.833333
3,red MacArthur,12047.333333
4,red Ashby,11595.500000
...,...,...
260,Customer 3297,0.000000
261,Customer 3535,0.000000
262,Customer 253,0.000000
263,Customer 900,0.000000


In [123]:
betweenness_df = betweenness_df[~betweenness_df['name'].str.contains('Customer|Store',regex = True)]

# create a station column since there are multiple nodes for the same station
betweenness_df['station'] = betweenness_df['name'].apply(lambda x: ' '.join(x.split(' ')[1:]))

# find the average betweenness for each station
grouped_station_betweenness = betweenness_df.groupby('station')['betweenness'].mean().reset_index()
grouped_station_betweenness.sort_values('betweenness')

/tmp/ipykernel_42/67468301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betweenness_df['station'] = betweenness_df['name'].apply(lambda x: ' '.join(x.split(' ')[1:]))


,station,betweenness
8,Berryessa,113.750000
4,Antioch,151.000000
32,OAK,151.000000
16,Dublin,151.000000
27,Millbrae,153.666667
40,SFO,223.250000
38,Richmond,231.500000
28,Milpitas,445.750000
35,Pittsburg Center,449.666667
48,West Dublin,606.333333


## Create New Store Locations

Create four new store locations and determine the shortest path from each store to each customer. See how different the mean travel times is.

In [124]:
#this block will create a new store node with all the corresponding relationships for the 4 stores.

create_new_store_with_relationships('Glen Park')
create_new_store_with_relationships('Richmond')
create_new_store_with_relationships('North Concord')
create_new_store_with_relationships('Bay Fair')

/tmp/ipykernel_42/1414378299.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_customers_df['nearest_store'] = f'Store {store_name}'
/tmp/ipykernel_42/1414378299.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_customers_df['nearest_store_lat_long'] = [store['lat_long']] * len(store_customers_df)
/tmp/ipykernel_42/1414378299.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

Determine the mean travel time for each of the different stores and only get ones that are over 40 minutes. North Concord is now the only station over 40 minutes.

In [125]:
stores = get_nodes(session, "Store")
customers = get_nodes(session, "Customer")
df = pd.DataFrame(columns=['store', 'customer', 'totalCost', 'nodes', 'costs'])
len_customers = len(customers)
# Loop through all stores and customers
for store in stores:
    for customer in customers:
        result = my_neo4j_shortest_path(store, customer)
        if result:
            df = df.append({
                'store': result['from'],
                'customer': result['to'],
                'totalCost': result['total_cost'],
                'nodes': result['nodes'],
                'costs': result['costs']
            }, ignore_index=True)

df['finalnode'] = df['nodes'].apply(lambda row: row[-2])
df = df.sort_values('totalCost',ascending=False)

min_cost_indices = df.groupby('customer')['totalCost'].idxmin()
min_cost_df = df.loc[min_cost_indices]

result_df = min_cost_df.groupby('finalnode')['totalCost'].agg(['mean','count']).reset_index()
print(result_df)
result_df.columns = ['finalnode','mean_travel_time_minutes','count_customers']
result_df['percent_customers'] = result_df['count_customers']/len(min_cost_df)*100
result_df = result_df.sort_values('mean_travel_time_minutes',ascending=False)
impacted_customers_df = result_df[result_df['mean_travel_time_minutes'] > 40]
impacted_customers_df

                     finalnode       mean  count
0                      Store 1  18.191111     10
1               Store Bay Fair  23.781481      3
2              Store Glen Park  20.325926      3
3          Store North Concord  41.333333      1
4               Store Richmond  25.511111      4
5   arrive 16th Street Mission  11.455556      2
6                 arrive Ashby   4.922222      1
7           arrive Balboa Park   7.533333      1
8          arrive Civic Center  11.588889      1
9              arrive Coliseum  31.011111      1
10              arrive Concord  17.600000      1
11            arrive Daly City  20.744444      1
12     arrive El Cerrito Plaza  12.366667      1
13          arrive Embarcadero  21.433333      4
14            arrive Fruitvale  21.641667      4
15         arrive Lake Merritt  24.111111      3
16            arrive MacArthur   8.577778      1
17       arrive North Berkeley  13.494444      2
18        arrive Pleasant Hill  21.611111      1
19        arrive Pow

,finalnode,mean_travel_time_minutes,count_customers,percent_customers
3,Store North Concord,41.333333,1,2.0
